# Questions per Linux Distribution
This notebook intends to show how many questions are under the tag for each linux distribution on serverfault.com.

# Step 1: get a list of programming languages
We will use the wikipedia page for [list of linux distributions](https://en.wikipedia.org/wiki/List_of_Linux_distributions) as our major source.

In [2]:
WIKIPEDIA_LIST_OF_LINUX_DISTRIBUTIONS = 'https://en.wikipedia.org/wiki/List_of_Linux_distributions'

import requests
from bs4 import BeautifulSoup

with requests.get(WIKIPEDIA_LIST_OF_LINUX_DISTRIBUTIONS) as r:
    soup = BeautifulSoup(r.text, 'html.parser')

Next, we scrape for names of linux distributions.

In [16]:
import re

table_rows = soup.find_all('tr')

linux_distros = []

for tr in table_rows:
    td = tr.find('td')
    if(td != None):
        a = td.find('a')
        if(a != None and a.text != ''):
            linux_distros.append(a.text)
            
for i in range(len(linux_distros)):
    linux_distros[i] = linux_distros[i].replace(' ', '-')

# Step 2: query serverfault for the number of questions having each tag

In [50]:
SERVERFAULT_TAG_SEARCH_URL_BASE = 'https://serverfault.com/questions/tagged/'

# simple class to hold data
class Distro:
    tag = ''
    num_questions = 0
    description = ''
    
    def __init__(self, tag, num_questions, description):
        self.tag = tag
        self.num_questions = num_questions
        self.description = description
        
    def __str__(self):
        return '{:10} {:10d} {:50.50}...'.format(self.tag, self.num_questions, self.description)
        
# regular expression for getting number from questions
re_num_questions = re.compile(r'(\d+,?)+' , re.MULTILINE)

linux_distro_stats = []
ld = 'ubuntu'

#for ld in linux_distros:
#    with requests.get(SERVERFAULT_TAG_SEARCH_URL_BASE + ld) as r:
with requests.get(SERVERFAULT_TAG_SEARCH_URL_BASE + ld) as r:
    if r.ok:
        ### create soup from webpage
        soup = BeautifulSoup(r.text, 'html.parser')
        
        ### find div with number of questions
        questions_text = soup.find('div', {'class': 'fs-body3 grid--cell fl1 mr12 sm:mr0 sm:mb12'})
        num_questions_match = re_num_questions.search(questions_text.text)
        num_questions_string = num_questions_match.group(0).replace(',', '')
        num_questions = int(num_questions_string)
        print(num_questions)
        
        ### find descripition
        description_container = soup.find_all('div', {'class': 'mb24'})
        # we are operating under the assumption that the description is the *second* div with 
        # class mb24. THIS MAY CHANGE without notice. We are subject to the whims of 
        # serverfault
        description_container = description_container[1]
        description = description_container.find('p').text
        print(description)
        
        ### put it all into a Distro object that gets added to our list
        linux_distro_stats.append(Distro(ld, num_questions, description))

print()
print(str(linux_distro_stats[0]))

14644

Ubuntu Linux is a Debian derivative that aims to bring Linux mainstream. Questions on this site should generally refer to Ubuntu Server. Note that http://askubuntu.com is specifically dedicated to Ubuntu questions.                    

ubuntu          14644 
Ubuntu Linux is a Debian derivative that aims to...
